In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import json
from sklearn.metrics import classification_report

## Part 1: Food Recognition Model

# Dataset configuration
# Dataset: https://www.kaggle.com/dansbecker/food-101
dataset_path = 'food-101'  # Main dataset directory
img_size = (224, 224)     # EfficientNet input size
batch_size = 32
epochs = 15

# Create data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Create generators
train_generator = train_datagen.flow_from_directory(
    os.path.join(dataset_path, 'images'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

val_generator = val_datagen.flow_from_directory(
    os.path.join(dataset_path, 'images'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

# Get class names and number of classes
class_names = list(train_generator.class_indices.keys())
num_classes = len(class_names)

print(f"Found {num_classes} food classes")

# Load pre-trained EfficientNet model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))

# Freeze base model layers
base_model.trainable = False

# Add custom head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile
model.compile(optimizer=Adam(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True),
    ModelCheckpoint('best_food_model.h5', save_best_only=True),
    ReduceLROnPlateau(factor=0.1, patience=2)
]

# Train
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    epochs=epochs,
    callbacks=callbacks)

# Evaluate
val_loss, val_acc = model.evaluate(val_generator)
print(f"\nValidation Accuracy: {val_acc:.2%}")

# Classification report
val_generator_no_shuffle = val_datagen.flow_from_directory(
    os.path.join(dataset_path, 'images'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False)

y_pred = np.argmax(model.predict(val_generator_no_shuffle), axis=1)
y_true = val_generator_no_shuffle.classes

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

## Part 2: Calorie Estimation

# Create a mock calorie database (in practice, you'd use a real nutrition database)
# Format: {'food_name': {'calories': X, 'serving_size': 'Yg'}}
calorie_db = {
    'apple_pie': {'calories': 300, 'serving_size': '100g'},
    'pizza': {'calories': 285, 'serving_size': '100g'},
    'steak': {'calories': 271, 'serving_size': '100g'},
    # Add all 101 classes with realistic values
}

# For demo purposes, we'll just fill missing values with 250 calories
for food in class_names:
    if food not in calorie_db:
        calorie_db[food] = {'calories': 250, 'serving_size': '100g'}

# Save the calorie database
with open('calorie_db.json', 'w') as f:
    json.dump(calorie_db, f)

# Function to recognize food and estimate calories
def recognize_food_and_calories(image_path, model, db_path='calorie_db.json'):
    # Load image
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=img_size)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    
    # Make prediction
    preds = model.predict(img_array)
    pred_class = class_names[np.argmax(preds)]
    confidence = np.max(preds)
    
    # Load calorie database
    with open(db_path) as f:
        calorie_db = json.load(f)
    
    # Get calorie info
    calorie_info = calorie_db.get(pred_class, {'calories': 250, 'serving_size': '100g'})
    
    # Display results
    plt.imshow(img)
    plt.title(f"Predicted: {pred_class}\nCalories: {calorie_info['calories']} per {calorie_info['serving_size']}\nConfidence: {confidence:.2%}")
    plt.axis('off')
    plt.show()
    
    return {
        'food': pred_class,
        'calories': calorie_info['calories'],
        'serving_size': calorie_info['serving_size'],
        'confidence': float(confidence)
    }

# Example usage
# result = recognize_food_and_calories('test_food.jpg', model)
# print(result)

## Part 3: Volume Estimation (Bonus)
# Note: This would require depth information or reference objects in the image
# Here's a conceptual placeholder function

def estimate_volume(image_path):
    """Placeholder for volume estimation functionality"""
    print("Volume estimation would require additional techniques like:")
    print("- Depth cameras or stereo imaging")
    print("- Reference objects in the image for scale")
    print("- 3D reconstruction techniques")
    return 1.0  # Default to 1 serving

# Complete system
def food_analysis_system(image_path):
    # Recognize food and get base calories
    food_info = recognize_food_and_calories(image_path, model)
    
    # Estimate volume (placeholder)
    serving_multiplier = estimate_volume(image_path)
    
    # Calculate total calories
    total_calories = food_info['calories'] * serving_multiplier
    
    return {
        **food_info,
        'estimated_servings': serving_multiplier,
        'total_calories': total_calories
    }

# Example usage
# analysis = food_analysis_system('test_food.jpg')
# print(analysis)